In [1]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import numpy as np
from keras.datasets import mnist
from keras import utils
from keras.optimizers import SGD
import cv2
from keras.layers.core import Activation, Flatten, Dense,Dropout

In [2]:
def VGG():
    model_vgg=VGG16(include_top=False,weights="imagenet",input_shape=(48,48,3))
    for layer in model_vgg.layers:
        layer.trainable=False
    model=Flatten(name="flatten")(model_vgg.output)
    model=Dense(10,activation="softmax")(model)
    vgg_mnist=Model(inputs=model_vgg.input,outputs=model,name="vgg16")


    return vgg_mnist

In [3]:
def train_mnist(dataset):
    
        (x_train, y_train), (x_test, y_test) = dataset.load_data() #original size train(xxx, 28, 28),test(xxx)
        
        #translate y_train and y_test to “one hot” form
        y_train = utils.to_categorical(y_train) 
        y_test = utils.to_categorical(y_test)

        #Modify the size of the data set and convert the grayscale image to an RGB image
        x_train=[cv2.cvtColor(cv2.resize(i,(48,48)),cv2.COLOR_GRAY2BGR)for i in x_train]  #x_train shape(10000,48,48,3)
        x_train=np.concatenate([arr[np.newaxis]for arr in x_train]).astype('float32')
        x_train=x_train/255
        x_test=[cv2.cvtColor(cv2.resize(i,(48,48)),cv2.COLOR_GRAY2BGR)for i in x_test]
        x_test=np.concatenate([arr[np.newaxis]for arr in x_test]).astype('float32')
        x_test=x_test/255
        
        model = VGG()
        sgd=SGD(lr=0.05,decay=1e-5)
        model.compile(optimizer=sgd,loss="categorical_crossentropy",metrics=['accuracy'])
        
        print("\nTraining：")
        model.fit(x_train, y_train, batch_size= 100, epochs= 5)

        # Evaluation
        print("\nEvaluation：")
        final_loss, final_accuracy = model.evaluate(x_test, y_test)
        print("loss= ", final_loss)
        print("accuracy= ", final_accuracy)
    
        
        print("success!")
        model.save('vgg_mnist.h5')
        
        outcome = model.predict(x_test[:1])
        print(outcome)

In [4]:
train_mnist(mnist)


Training：
Epoch 1/5
600/600 [==============================] - 3426s 6s/step - loss: 1.3035 - accuracy: 0.6460
Epoch 2/5
600/600 [==============================] - 3087s 5s/step - loss: 0.5477 - accuracy: 0.8636
Epoch 3/5
600/600 [==============================] - 3036s 5s/step - loss: 0.4290 - accuracy: 0.8914
Epoch 4/5
600/600 [==============================] - 3010s 5s/step - loss: 0.3743 - accuracy: 0.9044
Epoch 5/5
600/600 [==============================] - 2995s 5s/step - loss: 0.3373 - accuracy: 0.9123

Evaluation：
313/313 [==============================] - 505s 2s/step - loss: 0.3062 - accuracy: 0.9234
loss=  0.30624446272850037
accuracy=  0.9233999848365784
success!
[[6.9486259e-06 3.8848002e-04 8.5115893e-04 4.1322513e-05 8.1744062e-04
  7.3829222e-05 1.4947174e-04 9.9681109e-01 3.8630969e-05 8.2160172e-04]]
